In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_loisir(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    section = soup.find("div", id="sport-loisirs-tourisme")
    data = []
    
    if section:
        tables = section.find_all("table")
        for table in tables:
            rows = table.find_all("tr")
            for row in rows:
                cols = row.find_all(["td", "th"])
                if len(cols) >= 2:
                    label = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)
                    data.append({"Label": label, "Value": value})

    # Vérifie si on a des données
    if not data:
        print("❌ Aucun tableau trouvé ou structure vide")
        return None

    df_loisirs = pd.DataFrame(data)

    # Vérifie que la colonne 'Label' existe
    if "Label" not in df_loisirs.columns:
        print("❌ Colonne 'Label' manquante")
        return None

    # Supprime les lignes inutiles
    df_loisirs = df_loisirs[~df_loisirs["Label"].isin(["Données 2023", "Données 2024"])]
    
    return df_loisirs

In [30]:
# Lire par chunks
chunksize = 50
reader = pd.read_csv("communes-france-2025.csv", dtype={"code_insee": str}, low_memory=False, chunksize=chunksize)

data_concat = []

for chunk_index, df_chunk in enumerate(reader):
    # Supprimer première colonne si elle est inutile
    if df_chunk.columns[0].startswith("Unnamed"):
        df_chunk = df_chunk.drop(df_chunk.columns[0], axis=1)

    # Filtrer les villes > 20k habitants
    filtered_source = df_chunk[df_chunk["population"] >= 20000]

    for i in range(filtered_source.shape[0]):
        code_insee = filtered_source["code_insee"].iloc[i]
        ville = filtered_source["nom_sans_accent"].iloc[i]

        url = f"https://www.linternaute.com/ville/{ville}/ville-{code_insee}/sport-loisirs-tourisme"
        recherche = get_loisir(url)
        if recherche is not None:
            recherche["code_insee"] = code_insee
            recherche["Ville"] = ville
            data_concat.append(recherche)

            print("✅ Ligne ajoutée au DataFrame :")
            print(f"{ville} : {code_insee}")
            print("------------------------------------------------------------")

print("Recherches terminées")
# Concat final
df_final = pd.concat(data_concat, ignore_index=True)

loisirs = [
    "Nombre de théâtres", "Nombre de cinémas", 
    "Nombre de musées", "Nombre de conservatoires"
]

hotels = [
    "Nombre d'hôtel(s)", "Nombre de chambres d'hôtel", "Nombre de camping(s)",
    "Nombre d'emplacements de camping", "Hôtels non-classés", "Hôtels classés 1 étoile",
    "Hôtels classés 2 étoiles", "Hôtels classés 3 étoiles",
    "Hôtels classés 4 étoiles", "Hôtels classés 5 étoiles"
]

# On initialise la colonne
df_final["catégorie"] = df_final.iloc[:, 0].apply(
    lambda x: "Culture" if x in loisirs else "Hôtels" if x in hotels else "Sports"
)

df_final = df_final[df_final["Label"] != "Données 2021"]

df_final.to_csv("data_statut_pro.csv", index=False, encoding="utf-8-sig")

✅ Ligne ajoutée au DataFrame :
bourg-en-bresse : 01053
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
oyonnax : 01283
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
laon : 02408
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
saint-quentin : 02691
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
soissons : 02722
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
montlucon : 03185
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
vichy : 03310
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
manosque : 04112
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
gap : 05061
------------------------------------------------------------
✅ Ligne ajoutée au 

KeyboardInterrupt: 